# **Generate Korean DPR Dataset (KoAlpaca v1.1a)**

- KoAlpaca v1.1a 데이터 셋을 이용해 DPR 모델을 학습하기 위한 데이터 셋을 구축하는 코드입니다.
- 단답형 정답이 있는 데이터 셋은 **'generate_dataset_korquad.ipynb'** 파일을 참고해주세요. 
- Chunking은 nltk 라이브러리의 sent_tokenize 를 이용해 문장 단위로 이루어집니다. (여기서는 생략) 
- 이 코드로 구성한 최종 데이터 셋은 다음과 같은 조건을 만족해야 합니다. (코드를 따라가면 자동으로 충족됩니다.)
  - 모든 데이터 셋은 정해진 템플릿의 구성을 따라야 합니다.
  - 각 텍스트(text)는 고유한 인덱스 가져야 합니다.
  - Hard Negative 샘플을 이용할 경우 그 수는 텍스트의 수와 일치해야 합니다.

In [1]:
import json
import pandas as pd
from tqdm import tqdm
from nltk import sent_tokenize
from rank_bm25 import BM25Okapi
from datasets import load_dataset
from transformers import AutoTokenizer

### 1. Load Dataset

- KoAlpaca v1.1a 데이터 셋을 로드합니다.
- instruction, output 등의 정보를 확인합니다.
- 다른 데이터 셋을 이용할 경우 형식을 맞추거나 코드를 수정해서 사용하시면 됩니다.

In [3]:
dataset = load_dataset('beomi/KoAlpaca-v1.1a', trust_remote_code=True)

In [4]:
dataset['train'][0]

{'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?',
 'output': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.',
 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268'}

### 2. Chunk Contexts

- nltk의 sent_tokenize를 이용해 context를 chunk 단위로 분할합니다.
- **chunk_context** 함수에서 num_sents는 한 chunk 당 문장의 수이고, overlaps는 연속된 chunk 간 겹치는 문장의 수입니다.
- chunking 을 원하지 않을 경우 100 처럼 아주 큰 수를 입력하면 됩니다.
- 여기서는 chunking을 생략합니다. 큰 수를 입력하고 텍스트가 분할되지 않는지 확인합니다. 

In [5]:
def chunk_context(context, num_sents, overlaps):
    chunks = []
    start, end = 0, num_sents
    
    total_sents = sent_tokenize(context)

    while start < len(total_sents):
        chunk = total_sents[start:end]
        chunks.append(' '.join(chunk))
        
        start += (num_sents - overlaps)
        end = start + num_sents

    return chunks

In [6]:
sample_ctx = dataset['train'][0]['output']

chunks = chunk_context(sample_ctx, 100, 0)
chunks

['양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. 식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다. 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? 고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.']

### 3. Fill Templates

- 아래는 DPR 모델 학습을 위한 템플릿입니다.
- TEMPATE_ALL 의 positive 키의 리스트에 TEMPLATE_TEXT가 요소로 들어가는 구성입니다.
- __fill_templates__ 함수를 이용해 위 데이터 셋의 내용을 템플릿에 채웁니다. 
- 앞서 구성한 chunks와 아래의 question 이용해 템플릿을 잘 구성하는지 확인합니다.
- 사용하는 데이터 셋에 **제목이 없으면 그대로 "" 로 두면 됩니다**. None 값이 잘못 들어갈 경우 학습 과정에서 오류가 발생할 수 있습니다.  

In [7]:
TEMPLATE_ALL = {
    "question": "",
	"positive": [],
  }

TEMPLATE_TEXT = {
    "title": "",
    "text": "",
}

In [8]:
question = dataset['train'][0]['instruction']
question

'양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?'

In [9]:
def fill_templates(chunks, question):
    positives = []
    for chunk in chunks:
        template_instance1 = TEMPLATE_TEXT.copy()
        template_instance1['text'] = chunk
        positives.append(template_instance1.copy())

    template_instance2 = TEMPLATE_ALL.copy()
    template_instance2['question'] = question
    template_instance2['positive'] = positives
    
    return template_instance2

In [10]:
fill_templates(chunks, question)

{'question': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?',
 'positive': [{'title': '',
   'text': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. 식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다. 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? 고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.'}]}

### 4. Construct DPR Dataset

- 이상의 **chunk_context** 함수와 **fill_templates** 함수를 종합해 전체 데이터 셋에 적용합니다.
- 이를 수행하는 함수가 **construct_dpr_dataset** 입니다.
- 아래처럼 원하는 num_sents와 overlaps 값을 설정해 전체 데이터 셋에 함수를 적용합니다.

In [11]:
TEMPLATE_ALL = {
    "question": "",
	"positive": [],
  }

TEMPLATE_TEXT = {
    "title": "",
    "text": "",
}

In [12]:
def construct_dpr_dataset(dataset, num_sents, overlaps):
    dpr_dataset = []
    for idx in tqdm(range(len(dataset))):
        sample = dataset[idx]
        
        question = sample['instruction']
        chunks = chunk_context(sample['output'], num_sents, overlaps)
        
        output = fill_templates(chunks, question)
        dpr_dataset.append(output)
    
    return dpr_dataset

In [13]:
dpr_dataset = construct_dpr_dataset(dataset['train'], 100, 0)

100%|███████████████████████████████████████████████████████████████████████████| 21155/21155 [00:03<00:00, 5477.25it/s]


In [14]:
dpr_dataset[0]

{'question': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?',
 'positive': [{'title': '',
   'text': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. 식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다. 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? 고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.'}]}

- 코드 실행 결과, 위와 같은 완성된 템플릿이 담긴 리스트가 반환됩니다.
- **만일 여러 데이터 셋을 종합해서 사용할 경우, 이 단계에서 각 데이터 셋의 리스트를 하나로 통합해주면 됩니다.**
- 다음 단계에서 각 텍스트마다 고유한 인덱스를 부여하게 되므로 반드시 이 단계에서 통합해주어야 합니다. 

### 5. Set Index

- 데이터 셋의 각 텍스트에 고유한 인덱스를 부여합니다.
- 데이터 셋에 동일한 텍스트가 여러 개 있을 경우 모두 동일한 인덱스를 부여받게 됩니다.
- **text_2_index와 text_2_title** 은 **5.Add Hard Negative** 에서 이용합니다.

In [15]:
def set_index_to_text(dataset):
    text_2_index = {}
    text_2_title = {}
    current_idx = 0  # Initialize the starting index

    for sample in dataset:
        pos_text = [pos['text'] for pos in sample['positive']]
        pos_title = [pos['title'] for pos in sample['positive']]
    
        all_idx = []
        for text, title in zip(pos_text, pos_title):
            if text not in text_2_index:
                text_2_index[text] = current_idx 
                text_2_title[text] = title  # Map the text to its title
                current_idx += 1

        for pos in sample['positive']:
            pos['idx'] = text_2_index[pos['text']]
            all_idx.append(text_2_index[pos['text']])
        
        sample['answer_idx'] = all_idx

    return dataset, text_2_index, text_2_title

In [16]:
dpr_dataset, text_2_index, text_2_title = set_index_to_text(dpr_dataset)

In [17]:
dpr_dataset[0]

{'question': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?',
 'positive': [{'title': '',
   'text': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. 식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다. 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? 고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.',
   'idx': 0}],
 'answer_idx': [0]}

- 이상으로 DPR 모델을 학습할 수 있는 데이터 셋이 완성되었습니다.
- Hard Negative 샘플을 추가해 모델의 성능을 향상시키고 싶으면 **5.Add Hard Netgative** 로 가면됩니다.
- Hard Negative 샘플 없이 데이터를 저장하고 싶으면 **6.Save to Json** 으로 가면됩니다.
- **Validation Set의 경우 Hard Negative 샘플 없이 바로 저장하면 됩니다.**

### 5. Add Hard Negative (Optional)

- **get_hard_negative** 함수를 이용해 **'정답이 아니지만 정답처럼 보이는 텍스트(Hard Negative)'** 를 찾습니다. 
- BM25 점수가 가장 높은 **n** 개를 추출한 후 그 중에서 **가장 높은 점수**의 텍스트가 차례로 Hard Negative 샘플로 선택됩니다. 
- KorQuad의 경우 여러 질문이 동일한 텍스트에 연결된 경우가 많은데, False Negative를 방지하기 위해 동일한 텍스트는 모두 필터링합니다.
- 필터링의 결과로 n 개의 텍스트에서 Hard Negative 샘플을 정해진 수만큼 찾지 못한 경우 **기존의 것을 복제**해 사용합니다.
- 따라서 넉넉한 n 값을 설정하는 것이 좋습니다. 다만, n 값이 커질수록 데이터 셋 구축 시간이 오래걸립니다.
- 함수 실행 결과, **positive의 텍스트 개수와 hard_neg의 텍스트 개수는 일치해야 합니다**.  

In [18]:
def get_hard_negative(dataset, text_2_index, text_2_title, n=30):
    corpus = list(text_2_index.keys())
    tokenized_corpus = [context.split(" ") for context in corpus]
    bm25 = BM25Okapi(tokenized_corpus)

    for sample in tqdm(dataset, desc="Processing samples"):
        question = sample['question']
        positive = sample['positive']

        top_n = bm25.get_top_n(question.split(" "), corpus, n=n)  # Increase n to have more candidates
        
        hard_neg = []
        positive_idx_set = set(sample['answer_idx'])
        for doc in top_n:
            if text_2_index[doc] not in positive_idx_set:
                hard_neg.append({'title': text_2_title[doc],
                                 'text': doc,
                                 'idx': text_2_index[doc]})
            
            if len(hard_neg) >= len(positive):
                break
        
        # If not enough hard negatives, repeat existing ones to match the number of positives
        if len(hard_neg) < len(positive):
            print(f"Warning: Not enough hard negatives for question: {question}. Duplicating existing hard negatives.")
            while len(hard_neg) < len(positive):
                hard_neg.extend(hard_neg[:len(positive) - len(hard_neg)])
        
        sample['hard_neg'] = hard_neg[:len(positive)]  # Trim to ensure exact number matches positive samples
        
    return dataset

In [19]:
dpr_dataset = get_hard_negative(dpr_dataset, text_2_index, text_2_title, n=5)

Processing samples: 100%|█████████████████████████████████████████████████████████| 21155/21155 [19:33<00:00, 18.02it/s]


In [20]:
dpr_dataset[0]

{'question': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?',
 'positive': [{'title': '',
   'text': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. 식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다. 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? 고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.',
   'idx': 0}],
 'answer_idx': [0],
 'hard_neg': [{'title': '',
   'text': '양파는 백합과 관련된 식물 중 하나입니다. 백합과에는 양파뿐만 아니라 알리움, 파, 마늘, 부추 등도 포함됩니다. 모든 이들은 구근식물이며 이 식물들은 인경이라는 특별한 생물학적 특징을 가지고 있습니다. 인경은 줄기가 아닌 비늘 또는 덩어리 상태로 생장하는 식물을 지칭하는데, 양파도 구근이면서 인경이므로 백합 중 하나입니다. 또한 백합과는 구근 뿌리가 둥글다는 공통점이 있습니다.',
   'idx': 20722}]}

### 6. Save to Json

- 이상의 방법으로 구축한 데이터 셋을 json 파일로 저장합니다.
- 다시 로드해봄으로써 문제 없이 로드되는지 확인합니다.

In [21]:
file_path = 'koalpaca_v1_train.json'

In [22]:
with open(file_path, 'w') as outfile:
    json.dump(dpr_dataset, outfile, indent=4)

In [23]:
with open(file_path) as infile:
    dataset = json.load(infile)

In [24]:
dataset[0]

{'question': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?',
 'positive': [{'title': '',
   'text': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. 식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다. 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? 고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.',
   'idx': 0}],
 'answer_idx': [0],
 'hard_neg': [{'title': '',
   'text': '양파는 백합과 관련된 식물 중 하나입니다. 백합과에는 양파뿐만 아니라 알리움, 파, 마늘, 부추 등도 포함됩니다. 모든 이들은 구근식물이며 이 식물들은 인경이라는 특별한 생물학적 특징을 가지고 있습니다. 인경은 줄기가 아닌 비늘 또는 덩어리 상태로 생장하는 식물을 지칭하는데, 양파도 구근이면서 인경이므로 백합 중 하나입니다. 또한 백합과는 구근 뿌리가 둥글다는 공통점이 있습니다.',
   'idx': 20722}]}